In [ ]:
import os
import json
import numpy as np
import rasterio
from rasterio.windows import Window
from rasterio.transform import Affine
from rasterio.coords import BoundingBox
from rasterio.warp import transform_bounds
from PIL import Image

def save_sentinel_patches(
    tif_path,
    out_dir="data/sentinel_patches",
    patch_px=100,
    img_size=(224, 224),
    prefix="sentinel_patch"
):
    os.makedirs(out_dir, exist_ok=True)
    metadata = []
    patch_id = 0

    with rasterio.open(tif_path) as src:
        print(f"Opened GeoTIFF: {tif_path}")
        print(f"Image size: {src.width} x {src.height}")
        print(f"CRS: {src.crs}")

        # Use RGB bands if available (B4, B3, B2)
        band_indices = [4, 3, 2] if src.count >= 4 else [1, 2, 3]

        for row in range(0, src.height, patch_px):
            for col in range(0, src.width, patch_px):
                window = Window(col, row, patch_px, patch_px)

                # Skip partial tiles outside the bounds
                if (row + patch_px > src.height) or (col + patch_px > src.width):
                    continue

                # Read window
                patch = src.read(band_indices, window=window)
                if patch.shape[1] == 0 or patch.shape[2] == 0:
                    continue

                # Convert to uint8 RGB
                patch = np.transpose(patch, (1, 2, 0))
                patch = patch.astype(np.float32)
                patch /= np.percentile(patch, 98)  # Normalize brightness
                patch = np.clip(patch, 0, 1)
                patch = (patch * 255).astype(np.uint8)

                # Save resized patch
                img = Image.fromarray(patch)
                img = img.resize(img_size, Image.LANCZOS)
                img_path = os.path.join(out_dir, f"{prefix}_{patch_id}.png")
                img.save(img_path)

                # Compute geographic bounds (convert to WGS84 for readability)
                bounds = rasterio.windows.bounds(window, src.transform)
                bounds_wgs84 = transform_bounds(src.crs, "EPSG:4326", *bounds)
                minx, miny, maxx, maxy = bounds_wgs84
                centerx, centery = (minx + maxx) / 2, (miny + maxy) / 2

                # Store metadata
                metadata.append({
                    "patch_id": patch_id,
                    "bbox": {"minx": minx, "miny": miny, "maxx": maxx, "maxy": maxy},
                    "center": {"x": centerx, "y": centery},
                    "image_path": img_path.replace("\\", "/"),
                    "window_px": {"col": col, "row": row, "w": patch_px, "h": patch_px}
                })

                patch_id += 1

    # Save metadata
    json_path = os.path.join(out_dir, "patch_metadata.json")
    with open(json_path, "w") as f:
        json.dump(metadata, f, indent=4)

    print(f"✅ Saved {patch_id} patches")
    print(f"🗂 Metadata saved to {json_path}")
